# Machine Learning Portfolio Example

This example illustrates how to use all Chen-Zimmermann predictors, together with CRSP data. We'll merge monthly CRSP with the full set of Chen-Zimmermann predictors, fit the CRSP returns to lagged signals, and form portfolios in a super simple out-of-sample test. Specifically, we'll use a "groovy" model (fit on the 1960s and 1970s) to try to predict returns during hair metal (1980s), gangsta rap (1990s), and other more recent samples. Does the groovy model work even in the TSwift era?

Downloading all of the signals takes some time and requires substantial RAM. It also requires a WRDS account, since some predictors require data from WRDS (size, short-term reversal, price).

<span style="color:red;font-weight:bold">Note</span>

This example is an alternative version of *ML_portfolio_example.ipynb*. We prepared this example in case your computer has limited memory. The dataset used in this example is larger than 8GB, and Pandas DataFrame may struggle to process it if your computer does not have sufficient memory.
 
To address this, we use Polars DataFrame and set signal variables to float32 (instead of float64) to reduce memory usage.

The code should run without memory errors if your system has at least **16GB** of RAM.

In [1]:
import pandas as pd
import polars as pl
import openassetpricing as oap
import numpy as np
import wrds
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# initialize OpenAP
openap = oap.OpenAP()

# Download data

You'll have to enter your WRDS credentials twice: once to download the CRSP returns, and once to download all Chen-Zimmermann predictors (including size, short-term reversal, and price). The downloads take a couple minutes in total.

In [2]:
wrds_conn = wrds.Connection()

crsp = wrds_conn.raw_sql(
    """select permno, date, ret*100 as ret
                        from crsp.msf""",
    date_cols=["date"],
)

crsp = pl.from_pandas(crsp)

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
# download all Chen-Zimmermann predictors
bigdat = openap.dl_all_signals("polars")

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done

Data is downloaded: 4 mins


In [4]:
# get names of all signals
signal_list = [col for col in bigdat.columns if col not in ["permno", "yyyymm"]]

bigdat = bigdat.with_columns(pl.col(signal_list).cast(pl.Float32))

bigdat.head()

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M,Price,Size,STreversal
i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
10000,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.475906,-2.778819,-0.0
10000,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,-1.178655,-2.481568,0.257143
10000,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-1.490091,-2.793004,-0.365385
10000,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,-1.386294,-2.719452,0.098592
10000,198605,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.008817,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null,-1.134423,-2.467581,0.222656


# Lag signals and merge

To lag signals, you can just add one month to the `yyyymm` column for the signals. For simplicity, let's fill in the day of the new variable `date` as the 28th (the signals are assumed to be available for trading at the end of the month). You can keep around `yyyymm` as `yyyymm_signals` for sanity checks.

In [5]:
bigdat = bigdat.select(
    "permno",
    # create date that is one month ahead for merging with returns
    pl.col("yyyymm")
    .cast(pl.String)
    .add("28")
    .str.to_date("%Y%m%d")
    .dt.offset_by("1mo")
    .alias("date"),
    # rename yyyymm for clarity
    pl.col("yyyymm").alias("yyyymm_signals"),
    pl.col(signal_list),
)

bigdat.head()

permno,date,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M,Price,Size,STreversal
i32,date,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
10000,1986-02-28,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.475906,-2.778819,-0.0
10000,1986-03-28,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,-1.178655,-2.481568,0.257143
10000,1986-04-28,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-1.490091,-2.793004,-0.365385
10000,1986-05-28,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,-1.386294,-2.719452,0.098592
10000,1986-06-28,198605,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.008817,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null,-1.134423,-2.467581,0.222656


Now merge with CRSP. Convert CRSP dates to the 28th of the month for simplicity. The left join makes the missing values issues transparent.

In [6]:
# convert crsp dates to the 28th of the month
crsp = crsp.select(
    pl.col("permno").cast(pl.Int32),
    pl.col("date")
    .dt.year()
    .mul(100)
    .add(pl.col("date").dt.month())
    .cast(pl.String)
    .add("28")
    .str.to_date("%Y%m%d"),
    "ret",
)

# left join returns onto signals, in-place (for ram)
bigdat = crsp.join(bigdat, on=["permno", "date"], how="left")

bigdat.head()

permno,date,ret,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade12M,zerotrade1M,zerotrade6M,Price,Size,STreversal
i32,date,f64,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
10000,1985-12-28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
10000,1986-01-28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
10000,1986-02-28,-25.7143,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.475906,-2.778819,-0.0
10000,1986-03-28,36.5385,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,-1.178655,-2.481568,0.257143
10000,1986-04-28,-9.8592,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-1.490091,-2.793004,-0.365385


Congrats, the data is merged! But unfortunately, we'll need to do a bit more work to make it usable.

# Process data
We'll need to deal with the missing signals. This is a notorious issue with big data. Here, we'll just standardize the signals and then fill in missings with zero. This follows [Chen and McCoy (2024)](https://arxiv.org/abs/2207.13071).

In [ ]:
# copy over, keep only after 1963 and non-missing returns
cleandat = bigdat.filter(pl.col("date").dt.year() >= 1963, pl.col("ret").is_not_null())

cleandat = (
    cleandat
    # standardize
    .with_columns(
        (pl.col(signal_list) - pl.col(signal_list).mean()) / pl.col(signal_list).std()
    )
    # replace null with 0
    .fill_null(0)
)

# Form ML-style portfolios
Following Lewellen (2014, CFR), let's predict returns using many signals and then sort stocks on the predicted returns. We'll do this in perhaps the simplest way possible: fit returns with OLS using the "groovy" 1963-1979 sample. Then use the fitted coefficients on lagged signals to sort stocks every month from 1980 onward. This can't work, can it?

In [9]:
# user-specified fit period
fit_start = 1963
fit_end = 1979

# user-specified number of portfolios
nport = 5

cleandat_ols = cleandat

In [10]:
# fit returns
formula = "ret ~ " + " + ".join(signal_list)

fit = smf.ols(
    formula,
    data=cleandat_ols.filter(
        pl.col("date").dt.year().is_between(fit_start, fit_end - 1)
    ),
).fit()

In [11]:
# apply fit to all data
# do it chunk by chunk to avoid large momory consumption caused by large matrix operation
res = []
for i in cleandat_ols.iter_slices(n_rows=len(cleandat_ols) // 100):
    temp = i.select(pl.lit(1), pl.col(signal_list)).to_numpy() @ fit.params.values
    res += list(temp)

cleandat_ols = cleandat_ols.with_columns(pred=np.array(res))

In [12]:
# == find portfolio returns ==

# copy data
preddat = cleandat_ols.select("permno", "date", "pred", "ret").to_pandas()


# define port sort function
# follows https://github.com/chenandrewy/flex-mining/blob/70ca658090a13fea8517945280b2de83b9886968/0_Environment.R#L465
def port_sort(x, nport):
    return np.ceil(x.rank(method="min") * nport / (len(x) + 1)).astype(int)


preddat["port"] = preddat.groupby("date")["pred"].transform(port_sort, nport=nport)

# find portfolio returns
portdat = (
    preddat.groupby(["port", "date"], observed=False)
    .agg(ret=("ret", "mean"), nstock=("permno", "nunique"))
    .reset_index()
)

# Far Out-of-Sample Performance
Let's examine the performance of our groovy model, into the hair metal (1980s), gangsta rap (1990s), emo (2000s), EDM (2010s), and TSwift (2020s) samples.

In [13]:
# find performance by 10-year periods
samplength = 10

portdat["subsamp"] = pd.cut(
    portdat["date"].dt.year,
    bins=range(1959, 2030, samplength),
    labels=range(1959, 2029, samplength),
)

portsum = (
    portdat.groupby(["port", "subsamp"], observed=False)
    .agg(
        meanret=("ret", "mean"),
        vol=("ret", "std"),
        nmonth=("date", "nunique"),
        nstock=("nstock", "mean"),
        datemin=("date", "min"),
        datemax=("date", "max"),
    )
    .reset_index()
)
portsum["meanret"] = round(portsum["meanret"], 2)

# pivot meanret to wide format
sumwide = portsum.pivot(index="subsamp", columns="port", values="meanret").reset_index()
sumwide.columns = ["subsamp"] + [f"port_{col}" for col in sumwide.columns[1:]]

# add long-short
sumwide["5_minus_1"] = sumwide["port_5"] - sumwide["port_1"]

# add date ranges
temp = (
    portsum.groupby("subsamp", observed=False)
    .agg(datemin=("datemin", "min"), datemax=("datemax", "max"))
    .reset_index()
)

sumwide = pd.merge(temp, sumwide, on="subsamp", how="left")

# name the subsamples
sumwide["subsamp"] = sumwide["subsamp"].map(
    {
        1959: "groovy",
        1969: "groovy (still)",
        1979: "hair metal",
        1989: "gangsta rap",
        1999: "emo",
        2009: "EDM",
        2019: "TSwift",
    }
)

sumwide

,subsamp,datemin,datemax,port_1,port_2,port_3,port_4,port_5,5_minus_1
0,groovy,1963-01-28,1969-12-28,0.51,0.99,1.43,1.89,2.89,2.38
1,groovy (still),1970-01-28,1979-12-28,-0.81,0.35,1.03,1.83,3.24,4.05
2,hair metal,1980-01-28,1989-12-28,0.01,0.92,1.32,1.57,2.27,2.26
3,gangsta rap,1990-01-28,1999-12-28,0.09,0.67,1.08,1.55,3.31,3.22
4,emo,2000-01-28,2009-12-28,-0.18,0.51,0.80,1.07,2.32,2.50
5,EDM,2010-01-28,2019-12-28,0.48,0.72,0.73,0.85,1.24,0.76
6,TSwift,2020-01-28,2023-12-28,0.14,0.59,0.60,0.68,1.11,0.97


The model, fit only using groovy era data, makes it through hair metal, gansta rap, and emo quite well. In the corresponding decades, the groovy model earns long-short returns of 2.0 to 3.0 percent per month. So a model from the [Simon and Garfunkel](https://en.wikipedia.org/wiki/Groovy#/media/File:Soundofsilence.jpg) days continued to predict quite well, even while [Metallica inexplicably started to paint their fingernails black](https://www.reddit.com/r/Metallica/comments/huk18i/never_forget_emotallica/). During EDM and the Tswift eras, the model predicts with the some notable magnitudes, though the returns are much weaker than they were while [Ms. Swift was still into pickup trucks](https://www.youtube.com/watch?v=GkD20ajVxnY).

There are huge caveats about trading costs (Chen and Velikov 2023). But then again, this tutorial doesn't even attempt to deal with trading costs. One can likely do much better by following DeMiguel, Martin-Utrera, Nogales, and Uppal (2020) or Jensen, Kelly, Malamud, and Pedersen (2024).

# 3-Layer Neural Network Using scikit-learn

Let's replicate the steps above forming an ML-style portfolio, now with scikit-learn's implementation of a 3-layer neural network:

In [15]:
# User-specified parameters
fit_start = 1963
fit_end = 1979
nport = 5

cleandat_mlp = cleandat

In [16]:
# Prepare data for training
filtered_data = cleandat_mlp.filter(
    pl.col("date").dt.year().is_between(fit_start, fit_end - 1)
)
X = filtered_data.select(pl.col(signal_list)).to_pandas().values
y = filtered_data.select(pl.col("ret")).to_pandas().values.ravel()

In [17]:
# Standardize features for neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
# Train/test split for evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=1
)

# Define and train the MLPRegressor
mlp = MLPRegressor(
    hidden_layer_sizes=(64, 32, 16),  # Increase neurons for more learning capacity
    activation="relu",  # Good for nonlinear relationships
    solver="adam",  # Robust optimizer for noisy data
    alpha=0.001,  # Add regularization to reduce overfitting
    learning_rate_init=0.001,  # Lower learning rate for finer optimization
    max_iter=1000,  # Allow more iterations for convergence
    early_stopping=True,  # Stop training if validation error doesn't improve
    n_iter_no_change=20,  # Patience for early stopping
    random_state=1,  # Ensure reproducibility
)
mlp.fit(X_train, y_train)

MLPRegressor(alpha=0.001, early_stopping=True, hidden_layer_sizes=(64, 32, 16),
             max_iter=1000, n_iter_no_change=20, random_state=1)

In [23]:
# Apply the trained model to all data (chunk by chunk)
res = []
for i in cleandat_mlp.iter_slices(n_rows=len(cleandat_mlp) // 100):
    temp_data = i.select(pl.col(signal_list)).to_numpy()
    temp_data_scaled = scaler.transform(temp_data)
    temp_mlp = mlp.predict(temp_data_scaled)
    res.extend(temp_mlp)

cleandat_mlp = cleandat_mlp.with_columns(pred=np.array(res))

In [24]:
# == Find portfolio returns ==

# Copy data
preddat_mlp = cleandat_mlp.select("permno", "date", "pred", "ret").to_pandas()


# Define port sort function
def port_sort(x, nport):
    return np.ceil(x.rank(method="min") * nport / (len(x) + 1)).astype(int)


preddat_mlp["port"] = preddat_mlp.groupby("date")["pred"].transform(
    port_sort, nport=nport
)


# Find portfolio returns
portdat_mlp = (
    preddat_mlp.groupby(["port", "date"], observed=False)
    .agg(ret=("ret", "mean"), nstock=("permno", "nunique"))
    .reset_index()
)

In [25]:
# Find performance by 10-year periods
samplength = 10

portdat_mlp["subsamp"] = pd.cut(
    portdat_mlp["date"].dt.year,
    bins=range(1959, 2030, samplength),
    labels=range(1959, 2029, samplength),
)

portsum_mlp = (
    portdat_mlp.groupby(["port", "subsamp"], observed=False)
    .agg(
        meanret=("ret", "mean"),
        vol=("ret", "std"),
        nmonth=("date", "nunique"),
        nstock=("nstock", "mean"),
        datemin=("date", "min"),
        datemax=("date", "max"),
    )
    .reset_index()
)
portsum_mlp["meanret"] = round(portsum_mlp["meanret"], 2)

# Pivot meanret to wide format
sumwide_mlp = portsum_mlp.pivot(
    index="subsamp", columns="port", values="meanret"
).reset_index()
sumwide_mlp.columns = ["subsamp"] + [f"port_{col}" for col in sumwide_mlp.columns[1:]]

# Add long-short
sumwide_mlp["5_minus_1"] = sumwide_mlp["port_5"] - sumwide_mlp["port_1"]

# Add date ranges
temp_mlp = (
    portsum_mlp.groupby("subsamp", observed=False)
    .agg(datemin=("datemin", "min"), datemax=("datemax", "max"))
    .reset_index()
)

sumwide_mlp = pd.merge(temp_mlp, sumwide_mlp, on="subsamp", how="left")

# Name the subsamples
sumwide_mlp["subsamp"] = sumwide_mlp["subsamp"].map(
    {
        1959: "groovy",
        1969: "groovy (still)",
        1979: "hair metal",
        1989: "gangsta rap",
        1999: "emo",
        2009: "EDM",
        2019: "TSwift",
    }
)

sumwide_mlp

,subsamp,datemin,datemax,port_1,port_2,port_3,port_4,port_5,5_minus_1
0,groovy,1963-01-28,1969-12-28,0.03,0.75,1.37,2.04,3.51,3.48
1,groovy (still),1970-01-28,1979-12-28,-1.62,-0.13,1.00,2.14,4.25,5.87
2,hair metal,1980-01-28,1989-12-28,0.19,0.90,1.24,1.56,2.20,2.01
3,gangsta rap,1990-01-28,1999-12-28,0.39,0.61,1.05,1.50,3.14,2.75
4,emo,2000-01-28,2009-12-28,0.23,0.50,0.80,0.95,2.02,1.79
5,EDM,2010-01-28,2019-12-28,0.72,0.74,0.71,0.84,0.99,0.27
6,TSwift,2020-01-28,2023-12-28,0.54,0.64,0.48,0.61,0.84,0.30
